# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [41]:
infile = f"../data/tmp/1958.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [42]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [43]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [44]:
len(bigram_phrases.vocab.keys())

79825

Prenons une clé au hasard :

In [45]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

en_outre


Le dictionnaire indique le score de cette coocurrence :

In [46]:
bigram_phrases.vocab[key_]

19

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [47]:
%time bigram_phrases[sentences]

CPU times: user 5.82 ms, sys: 933 µs, total: 6.76 ms
Wall time: 11.1 ms


### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [48]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [49]:
%time bigram_phraser[sentences]

CPU times: user 7.33 ms, sys: 1.15 ms, total: 8.48 ms
Wall time: 14.3 ms


### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [50]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [51]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [52]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [53]:
print(corpus[:100])

[['ville'], [], ['de'], [], ['bruxelles'], [], ['rapport'], [], ['annuel'], [], ['presente'], ['a', 'u', 'conseil', 'c', 'o', 'm', 'm', 'u', 'n', 'a', 'l', 'e', 'n', 'seance', 'd', 'u', '6', 'o', 'c', 't', 'o', 'b', 'r', 'e', '1958'], ['par', 'l', 'e', 'c', 'o', 'l', 'l', 'e', 'g', 'e', 'des', 'bourgmestre', 'e', 't', 'echevins'], ['en', 'execution', 'de', 'l', "'", 'article', '70', 'de_la_loi', 'du', '30', 'mars', '1836'], [], ['imprimerie', 'h', '.', '&', 'm', '.', 'schaumans', ',', 's', '.', 'a', '.'], ['41', ',', 'parvis', 'saint_-', 'gilles'], ['bruxelles'], [], ['1958'], [], ['chapitre'], [], ['premier'], [], ['administration'], [], ['chapitre'], [], ['premier'], [], ['administration'], ['l', 'e', 'conseil', 's', "'", 'est', 'reuni', '60', 'fois', 'au_cours', 'de', 'l', "'", 'annee_1957', '.'], ['il', 'a', 'tenu', 'en_outre', '18', 'seances', 'des', 'sections', 'reunies', ',', '152', 'seances', 'des', 'sections', 'et', '25', 'seances', 'de', 'commissions', 'ou', 'comites', 'diver

## Entrainement d'un modèle Word2Vec sur ce corpus

In [54]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 3.09 s, sys: 112 ms, total: 3.2 s
Wall time: 2.13 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [55]:
outfile = f"../data/1958.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [56]:
model = Word2Vec.load("../data/1958.model")

### Imprimer le vecteur d'un terme

In [57]:
model.wv["enseignement"]

array([-0.37944463, -1.27537   ,  0.5021252 ,  0.43074915, -0.01694473,
       -0.21129358,  0.5578967 ,  0.09498172,  0.2924875 ,  0.31869748,
       -0.05453331,  0.30779603,  0.33572415, -0.59399444, -0.08055618,
       -0.19021785, -0.7088651 ,  0.35172692,  0.01765961,  0.6837619 ,
        0.68366796,  0.6762323 ,  0.8476853 , -0.23448749,  0.49296   ,
        0.14884743, -0.3364861 , -0.42259204,  0.94019336, -0.56744736,
       -0.20436105, -0.04335768], dtype=float32)

## Calculer la similarité entre deux termes

In [58]:
model.wv.similarity("enseignement", "eleves")

0.91881955

### Chercher les mots les plus proches d'un terme donné

In [59]:
model.wv.most_similar("enseignement", topn=10)

[('etat', 0.9920451641082764),
 ('application', 0.9806413650512695),
 ('exposition', 0.9795581102371216),
 ('locaux', 0.9716431498527527),
 ('pas', 0.969896674156189),
 ('au_cours', 0.9684438109397888),
 ('administration', 0.968135416507721),
 ('appel', 0.966103732585907),
 ('etude', 0.9642866849899292),
 ('qui', 0.9631277322769165)]

### Faire des recherches complexes à travers l'espace vectoriel

In [ ]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))